In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('/content/drive/MyDrive/Machine Learning (ITHUB) /ДИ 2023/lessons/data/vacancies.xlsx')
df.head()

,title,description,real_salary
0,Программист C# (Senior),"Agile. Azure DevOps (build server, task tracke...",97500.0
1,Начинающий программист,"Работаем с реляционными БД, поэтому желательно...",50000.0
2,Начинающий программист,"Работаем с реляционными БД, поэтому желательно...",50000.0
3,Инженер - программист 1С,Опыт работы с конфигурацией 1С ERP2 и знание о...,70000.0
4,Программист кастомной конфигурации 1С (Middle),Поддержка управленческой самописной БД в 18...,120000.0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2421 entries, 0 to 2704
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        2421 non-null   object 
 1   description  2421 non-null   object 
 2   real_salary  2421 non-null   float64
dtypes: float64(1), object(2)
memory usage: 140.2+ KB


In [22]:
df['description'].values[:10]

array(['Agile. Azure DevOps (build server, task tracker, git repo, documentation, web hooks). Git + git flow. .NET Framework 4.5... Разработка и доработка ключевых продуктов и сервисов. Решение наиболее сложных и ответственных задач. Активное участие в планировании работы команды. ',
       'Работаем с реляционными БД, поэтому желательно понимание принципов их работы и умение оптимизировать логику обработки данных – SQL, MySQL, PostgreSQL.  ...доработка интерфейсной части и отчетных форм, внесение изменений в бизнес-логику, реализация модулей для взаимодействия с внешними системами. ',
       'Опыт работы с конфигурацией 1С ERP2 и знание основ бухгалтерского учета приветствуется. Опыт работы в должности инженера-программиста... Сопровождение 1С УПП. Разработка нового функционала 1С УПП. Консультация пользователей. Участие во внедрении 1С ERP. ',
       '   Поддержка управленческой самописной БД в 18 ТБ; Поддержка обмена между головным и дочерним узлом 2 млн объектов в сутки; Ежедневые 

In [3]:
df.drop_duplicates(keep='first',inplace=True)
df.head()

,title,description,real_salary
0,Программист C# (Senior),"Agile. Azure DevOps (build server, task tracke...",97500.0
1,Начинающий программист,"Работаем с реляционными БД, поэтому желательно...",50000.0
3,Инженер - программист 1С,Опыт работы с конфигурацией 1С ERP2 и знание о...,70000.0
4,Программист кастомной конфигурации 1С (Middle),Поддержка управленческой самописной БД в 18...,120000.0
5,Fullstack web-программист,"Умение разбираться в поставленных задачах, уто...",165000.0


Задачей синтаксического анализа является определение связей между словами. Результатом синтаксического анализа является дерево одного из двух видов - дерево зависимостей или дерево составляющих. Дерево зависимостей показывает связи между отдельными словами, дерево составляющих показывает как слова складываются в группы. Так как дерево зависимостей содержит больше информации, мы будем использовать именно этот формализм.

In [4]:
!pip install ufal.udpipe

## UDPipe

* Помимо этого, нам потребуются языковые синтаксические модели, взятые отсюда https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-3131
* Более конкретно, эту (уже есть в вашей папке data)
https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/russian-syntagrus-ud-2.5-191206.udpipe?sequence=70&isAllowed=y

In [5]:
# Подключаем синтаксической анализатор и его части.
import ufal.udpipe
from ufal.udpipe import Model, Pipeline, ProcessingError

# Для отрисовки деревьев нам потребуются вот эти библиотеки.

from collections import defaultdict
from tqdm.auto import tqdm
import random

In [6]:
#загружаем модель
model = Model.load("/content/drive/MyDrive/Machine Learning (ITHUB) /ДИ 2023/lessons/data/russian-syntagrus-ud-2.5-191206.udpipe")

In [7]:
# Токенизатор со снятием омонимии.
tokenizer = model.newTokenizer(model.DEFAULT)
# Объект для отображения результатов разбора.
conlluOutput = ufal.udpipe.OutputFormat.newOutputFormat("conllu")
# Объект предложения, в которое будет осуществляться разбор.
sentence = ufal.udpipe.Sentence()
# Объект для ошибок, возникающих в ходе разбора.
error = ufal.udpipe.ProcessingError()

In [15]:
df['description'][5]

'Умение разбираться в поставленных задачах, уточнять их условия и результат, выбирать эффективные способы решения. Понимание ООП и паттернов проектирования, следование... Мы работаем над предоставлением нашим клиентам целого комплекса IT-решений, начиная с e-commerce и заканчивая digital-трансформацией традиционной розницы. '

In [16]:
#кормим токенизатору наш текст
tokenizer.setText(df['description'][5])

Далее предложение передается на морфологическую разметку со снятием омонимии (тэггинг) и синтаксический анализ (парсинг).

Результаты разбора можно отображать в разных форматах, но мы будем использовать формат conllu.

In [17]:
tokenizer.nextSentence(sentence, error)
model.tag(sentence, model.DEFAULT)
model.parse(sentence, model.DEFAULT)
print(conlluOutput.writeSentence(sentence))

# sent_id = 3
# text = Умение разбираться в поставленных задачах, уточнять их условия и результат, выбирать эффективные способы решения.
1	Умение	умение	NOUN	_	Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing	0	root	_	_
2	разбираться	разбираться	VERB	_	Aspect=Imp|VerbForm=Inf|Voice=Mid	1	obl	_	_
3	в	в	ADP	_	_	5	case	_	_
4	поставленных	ставить	VERB	_	Aspect=Perf|Case=Loc|Number=Plur|Tense=Past|VerbForm=Part|Voice=Pass	5	amod	_	_
5	задачах	задача	NOUN	_	Animacy=Inan|Case=Loc|Gender=Fem|Number=Plur	2	obl	_	SpaceAfter=No
6	,	,	PUNCT	_	_	7	punct	_	_
7	уточнять	уточнять	VERB	_	Aspect=Imp|VerbForm=Inf|Voice=Act	2	conj	_	_
8	их	их	DET	_	_	9	det	_	_
9	условия	условие	NOUN	_	Animacy=Inan|Case=Acc|Gender=Neut|Number=Plur	7	obj	_	_
10	и	и	CCONJ	_	_	11	cc	_	_
11	результат	результат	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	9	conj	_	SpaceAfter=No
12	,	,	PUNCT	_	_	13	punct	_	_
13	выбирать	выбирать	VERB	_	Aspect=Imp|VerbForm=Inf|Voice=Act	2	conj	_	_
14	эффективные	эффективный	ADJ	_	Animacy=Ina

Объект Sentence содержит в себе свойство words, являющееся списком слов. Каждое слово включает в себя, среди прочего, следующие свойства:

* lemma - начальная форма слова,
* form - токен,
* head - номер родительского элемента в списке слов,
* deprel - название отношения,
* upostag - часть речи,
* feats - грамматические параметры,
* misc - дополнительные параметры.

In [18]:
df_sentence = pd.DataFrame(columns=['head','lemma','form','upostag','feats','deprel','misc'])

In [19]:
for i, word in enumerate(sentence.words):
  df_sentence.loc[len(df_sentence)] = [word.head, word.lemma, word.form, word.upostag, word.feats, word.deprel, word.misc]
    #print(i, "->", word.head, word.lemma, "[", word.form, "]", word.upostag, word.feats, word.deprel, word.misc)
df_sentence

,head,lemma,form,upostag,feats,deprel,misc
0,-1,<root>,<root>,<root>,<root>,,
1,0,умение,Умение,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing,root,
2,1,разбираться,разбираться,VERB,Aspect=Imp|VerbForm=Inf|Voice=Mid,obl,
3,5,в,в,ADP,,case,
4,5,ставить,поставленных,VERB,Aspect=Perf|Case=Loc|Number=Plur|Tense=Past|Ve...,amod,
5,2,задача,задачах,NOUN,Animacy=Inan|Case=Loc|Gender=Fem|Number=Plur,obl,SpaceAfter=No
6,7,",",",",PUNCT,,punct,
7,2,уточнять,уточнять,VERB,Aspect=Imp|VerbForm=Inf|Voice=Act,conj,
8,9,их,их,DET,,det,
9,7,условие,условия,NOUN,Animacy=Inan|Case=Acc|Gender=Neut|Number=Plur,obj,


Давайте возьмем текст и выделим из него все сочетания "существительное + существительное в родительном падеже". Такие сочетания интересны тем, что могут претендовать на роль терминов.

In [23]:
def find_child_Ng(sentence, parent):
    """ Функция поиска всех потомков вершины с номером parent, являющихся
    существительными в родительном падеже.
    """
    children = []
    for i, word in enumerate(sentence.words):
        if word.head == parent and word.upostag == "NOUN" and "Case=Gen" in word.feats:
            children.append(i)
    return children

def find_NNg(sentence, combinations):
    """ Функция поиска сочетаний "существительное + существительное в родительном падеже".
    """
    for i, word in enumerate(sentence.words):
        if word.upostag == "NOUN":
            children = find_child_Ng(sentence, i)
            if children != []:
                #print(word.lemma, [sentence.words[k].form for k in children])
                for child in children:
                    combinations[word.lemma][sentence.words[child].form] += 1

In [24]:
combinations = defaultdict(lambda:defaultdict(int))

for text in tqdm(df['description'][:1000]):
    tokenizer.setText(text) # Установить текст.
    while tokenizer.nextSentence(sentence, error): # Полчаем следующее предложение.
        model.tag(sentence, model.DEFAULT) # Токенизация.
        model.parse(sentence, model.DEFAULT) # Синтаксический анализ.
        find_NNg(sentence, combinations) # Поиск сочетаний.

  0%|          | 0/1000 [00:00<?, ?it/s]

In [26]:
df_nn=pd.DataFrame(columns=['parent','children','freq'])
for parent, childs in combinations.items():
    for child, freq in childs.items():
        if freq > 20:
            df_nn.loc[len(df_nn)] = [parent, child,freq]

In [27]:
df_nn.sort_values(by='freq',ascending=False).head(20)

,parent,children,freq
5,опыт,работы,404
6,опыт,разработки,136
24,язык,программирования,73
11,знание,языка,65
8,опыт,программирования,62
26,проведение,тренировок,56
22,база,данных,51
16,разработка,обеспечения,50
10,знание,основ,49
2,работа,лет,41
